In [146]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd 
import sqlite3
import numpy as np
from datetime import datetime

url = "https://en.wikipedia.org/wiki/List_of_largest_banks"
db_name = 'bank_db'
table_name = 'bank_table'
file_path = 'bank_gdp.csv'
conn = sqlite3.connect(db_name)

def extract(url):
     
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.content, 'html.parser')
    df = pd.DataFrame(columns = ['Global_rank','Bank_name', 'Market_cap_in_USD'])

    tables = soup.find_all('tbody')


    rows = tables[0].find_all('tr')
    #print(rows[0])

    for row in rows:
            col = row.find_all('td')
            #print(col)
            if len(col) != 0:
                if col[1].find_all('a') is not None:
                    data_dict = {'Global_rank' : col[0].contents[0],
                                'Bank_name': col[1].find_all('a')[1].contents,
                                'Market_cap_in_USD': col[2].contents[0]}
                    df1 = pd.DataFrame(data_dict, index = [1])
                    df = pd.concat([df,df1], ignore_index= True)

    df = df.replace('\n','', regex = True ) 
    return df


def transform(df):
    dataframe = pd.read_csv("exchange_rate.csv")
    dict = dataframe.set_index('Currency').to_dict()['Rate']
    #dict['EUR']
    GDP_list = df['Market_cap_in_USD'].tolist()
    #print(GDP_list)
    GDP_list = [float("".join(x.split())) for x in GDP_list]
    #print(GDP_list)
    df['Market_cap_in_EUR_BILLION'] = [np.round(x*dict['EUR'],2) for x in GDP_list]
    df['Market_cap_in_GBP_BILLION'] = [np.round(x*dict['GBP'],2) for x in GDP_list]
    df['Market_cap_in_INR_BILLION'] = [np.round(x*dict['INR'],2) for x in GDP_list]
    return df


def load_csv(file_path, df):
     
     df.to_csv(file_path)

def load_db(df,table_name, conn):
     
     df.to_sql(table_name, conn, if_exists = 'replace', index = False)


def run_query(table_name, conn):

    query_statement_1 = f"SELECT * FROM {table_name}"
    #print(query_statement_1)
    query_output_1 = pd.read_sql(query_statement_1, conn)
    #print(query_output_1)

    query_statement_2 = f"SELECT AVG(Market_cap_in_GBP_BILLION) FROM {table_name}"
    print(query_statement_2)
    query_output_2 = pd.read_sql(query_statement_2, conn)
    print(query_output_2)

    query_statement_3 = f"SELECT * from {table_name} LIMIT 5"
    #print(query_statement_3)
    query_output_3 = pd.read_sql(query_statement_3, conn)
    print(query_output_3)


def log_progress(message): 

    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("bank_gdp_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')


log_progress('Preliminaries complete. Initiating ETL process')
df = extract(url)
log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df)
log_progress('Data transformation complete. Initiating loading process')

load_csv(file_path, df)
log_progress('Data saved to CSV file')


log_progress('SQL Connection initiated.')
load_db(df,table_name, conn)

log_progress('Data loaded to Database as table. Running the query')

run_query(table_name, conn)
log_progress('Process Complete.')
conn.close()

#print(df)

Mzk5My1rYXNoeWFw

#print(df)



        #print(col)
     


SELECT AVG(Market_cap_in_GBP_BILLION) FROM bank_table
   AVG(Market_cap_in_GBP_BILLION)
0                         151.987
  Global_rank                                Bank_name Market_cap_in_USD  \
0           1                           JPMorgan Chase            432.92   
1           2                          Bank of America            231.52   
2           3  Industrial and Commercial Bank of China            194.56   
3           4               Agricultural Bank of China            160.68   
4           5                                HDFC Bank            157.91   

   Market_cap_in_EUR_BILLION  Market_cap_in_GBP_BILLION  \
0                     402.62                     346.34   
1                     215.31                     185.22   
2                     180.94                     155.65   
3                     149.43                     128.54   
4                     146.86                     126.33   

   Market_cap_in_INR_BILLION  
0                   35910.71  
1   

In [141]:
dataframe = pd.read_csv("exchange_rate.csv")
dict = dataframe.set_index('Currency').to_dict()['Rate']
dict['EUR']
GDP_list = df['Market_cap_in_USD'].tolist()
#print(GDP_list)
GDP_list = [float("".join(x.split())) for x in GDP_list]
print(GDP_list)
df['Market_cap_in_EUR_BILLION'] = [np.round(x*dict['EUR'],2) for x in GDP_list]
df['Market_cap_in_GBP_BILLION'] = [np.round(x*dict['GBP'],2) for x in GDP_list]
df['Market_cap_in_INR_BILLION'] = [np.round(x*dict['INR'],2) for x in GDP_list]
#df = df.drop([''])
df.to_csv(file_path)

df.to_sql(table_name, conn, if_exists = 'replace', index = False)

query_statement_1 = f"SELECT * FROM {table_name}"
print(query_statement_1)
query_output_1 = pd.read_sql(query_statement_1, conn)
print(query_output_1)

query_statement_2 = f"SELECT AVG(Market_cap_in_USD) FROM {table_name}"
print(query_statement_2)
query_output_2 = pd.read_sql(query_statement_2, conn)
print(query_output_2)

query_statement_3 = f"SELECT Bank_name from {table_name} LIMIT 5"
print(query_statement_3)
query_output_3 = pd.read_sql(query_statement_3, conn)
print(query_output_3)







[432.92, 231.52, 194.56, 160.68, 157.91, 155.87, 148.9, 140.83, 139.82, 136.81]
SELECT * FROM bank_table
  Global_rank                                Bank_name Market_cap_in_USD  \
0           1                           JPMorgan Chase            432.92   
1           2                          Bank of America            231.52   
2           3  Industrial and Commercial Bank of China            194.56   
3           4               Agricultural Bank of China            160.68   
4           5                                HDFC Bank            157.91   
5           6                              Wells Fargo            155.87   
6           7                        HSBC Holdings PLC            148.90   
7           8                           Morgan Stanley            140.83   
8           9                  China Construction Bank            139.82   
9          10                            Bank of China            136.81   

   Market_cap_in_EUR_BILLION  Market_cap_in_GBP_BILLION  \

In [ ]:
for row in rows:
    if row.find_all('td') is not None:
        
        col = row.find_all('td')
        #print(col)
        if len(col) != 0:
            if col[1].find('a') is not None:
                #print(col)
                for c in col:
                    coll = c.find_all('a')
                    print(coll)
                    #data = {'name': coll[1].contents}
    else:
        pass